In [67]:
import numpy as np
import pandas as pd
import pickle
from urllib.parse import urlparse
from nltk.corpus import stopwords


from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
from tensorflow.keras import preprocessing as tfkp
from tensorflow.keras.utils import to_categorical

import gensim.downloader as api

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ben
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
import string

def load_data():
    filename = "../data/twitter_data_fixed.pkl"
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    ids = []
    for i in range(len(data)):
        text = "".join(l for l in data[i]['text'] if l not in string.punctuation)
        x_text.append((data[i]['text']).encode('utf-8'))
        labels.append(data[i]['label'])
    return x_text,labels

In [5]:
def is_url(url):
  try:
    result = urlparse(url)
    return all([result.scheme, result.netloc])
  except ValueError:
    return False

In [145]:
# embedding stuff

def map_weights(embed_dict, vocab, embed_size): # embed size is embedding dim
    vocab_size = len(vocab) + 1
    weights = np.zeros((vocab_size, embed_size))

    n_missed = 0
    words_missed = []
    for k,v in vocab.items():
        try:
            weights[v] = embed_dict[k]  # weights[v] is an index, embed_dict[k] is the list of weights
        except:
            n_missed += 1
            words_missed.append(k)
    print(f"{n_missed} embeddings missed of {vocab_size}")
    return weights, words_missed

### Loading Data

In [7]:
x_text, labels_og = load_data()
labels, uniques = pd.factorize(labels_og)

Loading data from file: ../data/twitter_data_fixed.pkl


### Intitial Twitter-Specific Pre-processing

In [8]:
comments = pd.DataFrame({'comment': x_text, 'attack': labels})

# decode to UTF-8
comments['comment'] = comments['comment'].str.decode("utf-8")

# remove missing rows
comments['comment'].dropna(inplace=True)

# remove usernames
comments['comment'] = comments['comment'].str.replace('(\@\w+.*?)',"", regex=True)

# lower case everything
comments['comment'] = comments['comment'].str.lower()

# remove URLs
comments['comment'] = [' '.join(y for y in x.split() if not is_url(y)) for x in comments['comment']]

# remove stop words
comments['comment'] = comments['comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))

# traditionally, would also lemmatize but this was not done in the main data

In [9]:
comments.head(n=100)

,comment,attack
0,rt : another bloody instant restaurant week ? ...,0
1,video peshmerga decimating isis far interesting.,0
2,oh really ? instant restaurants ? that's shock...,0
3,rt : good weeks #isis. new front opened #sinja...,0
4,rt : don’t need femisnsn men carry heavy thing...,0


### Train-Test Split and Tokenization

In [11]:
X_train_pretoken, X_midway_pretoken, y_train, y_midway = train_test_split(comments['comment'], comments['attack'], random_state = 42, test_size=0.2)

In [12]:
dim = 25
n_classes = len(np.unique(y_train.values))

tokenizer = tfkp.text.Tokenizer(oov_token="<UNK>", filters='!"$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',)
tokenizer.fit_on_texts(X_train_pretoken)

convert = lambda x: tfkp.sequence.pad_sequences(tokenizer.texts_to_sequences(x),
                                                    maxlen=dim,
                                                    padding='post', truncating='post')

X_train = convert(X_train_pretoken)
X_midway = convert(X_midway_pretoken)

In [13]:
X_test, X_val, y_test, y_val = train_test_split(X_midway, y_midway, random_state = 42, test_size=0.5)

### Oversampling

In [17]:
oversample = RandomOverSampler(random_state=42)
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

### Reshaping for input

In [69]:
y_train_onehot = to_categorical(y_train, n_classes)
y_val_onehot = to_categorical(y_val, n_classes)
y_test_onehot = to_categorical(y_test, n_classes)
y_train_over_onehot = to_categorical(y_train_over, n_classes)

### Glove Embedding Weights

In [142]:
glove_dict = api.load("glove-twitter-200")
glove_weights, glove_words_missed = map_weights(glove_dict, tokenizer.word_index, 200)

Loading data from file: glove_embeddings/glove.twitter.27B.200d.txt


FileNotFoundError: [Errno 2] No such file or directory: 'glove_embeddings/glove.twitter.27B.200d.txt'

### Word2vec Weights

In [157]:
word_dict = api.load("word2vec-google-news-300")
word2vec_weights, wrod2vec_words_missed = map_weights(word_dict, tokenizer.word_index, 300)

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

#### Checks

In [158]:
print(np.shape(X_train))
print(np.shape(X_train_over))
print(np.shape(X_val))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_train_over))
print(np.shape(y_val))
print(np.shape(y_test))
print(np.shape(glove_weights))

(12872, 25)
(26508, 25)
(1609, 25)
(1609, 25)
(12872,)
(26508,)
(1609,)
(1609,)
(15155, 200)


In [136]:
print(np.shape(y_train_over_onehot))

(26508, 3)


In [21]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_train_over))

Counter({0: 8836, 2: 2503, 1: 1533})
Counter({2: 8836, 0: 8836, 1: 8836})


### Write Embedders

In [159]:
from keras.layers import Embedding
from tensorflow.keras import initializers

vocab_size = len(tokenizer.word_index)+1

random_embedder = Embedding(vocab_size, 200, input_length=dim, trainable=True)
glove_embedding = Embedding(vocab_size, 200, embeddings_initializer=initializers.Constant(glove_weights),
                            trainable=False)
word2vec_embedding = Embedding(vocab_size, 300, embeddings_initializer=initializers.Constant(word2vec_weights),
                              trainable=False)

### Load DL Models and Run Them

In [114]:
from DL_models import lstm_keras, cnn_keras, blstm, blstm_atten

In [162]:
lstm_random = lstm_keras(dim,n_classes,random_embedder)
lstm_random_over = lstm_keras(dim,n_classes,random_embedder)
lstm_glove = lstm_keras(dim,n_classes,glove_embedding)
lstm_glove_over = lstm_keras(dim,n_classes,glove_embedding)

In [128]:
blstm_random = blstm(dim_,n_classes,random_embedder)
blstm_random_over = blstm(dim,n_classes,random_embedder)
blstm_glove = blstm(dim,n_classes,glove_embedding)
blstm_glove_over = blstm(dim,n_classes,glove_embedding)

In [140]:
#BLSTM and CNN need work

blstm_atten_random = blstm_atten(dim,n_classes,random_embedder)
blstm_atten_random_over = blstm_atten(embed_size,n_classes,random_embedder)
blstm_atten_glove = blstm_atten(embed_size,n_classes,glove_embedding)
blstm_atten_glove_over = blstm_atten(embed_size,n_classes,glove_embedding)

ValueError: All layers in a Sequential model should have a single output tensor. For multi-output layers, use the functional API.

In [138]:
cnn_random = cnn_keras(embed_size,n_classes,random_embedder)
cnn_random_over = cnn_keras(embed_size,n_classes,random_embedder)
cnn_glove = cnn_keras(embed_size,n_classes,glove_embedding)
cnn_glove_over = cnn_keras(embed_size,n_classes,glove_embedding)

ValueError: Exception encountered when calling layer "max_pooling1d_26" (type MaxPooling1D).

Negative dimension size caused by subtracting 35 from 1 for '{{node max_pooling1d_26/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 35, 1, 1], padding="VALID", strides=[1, 35, 1, 1]](max_pooling1d_26/ExpandDims)' with input shapes: [?,1,1,64].

Call arguments received:
  • inputs=tf.Tensor(shape=(None, 1, 64), dtype=float32)

In [163]:
lstm_random.summary()
lstm_random.fit(X_train,  y_train_onehot, epochs=3, validation_data=(X_val, y_val_onehot))

Model: "sequential_73"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 25, 200)           3031000   
                                                                 
 dropout_146 (Dropout)       (None, 25, 200)           0         
                                                                 
 lstm_61 (LSTM)              (None, 25)                22600     
                                                                 
 dropout_147 (Dropout)       (None, 25)                0         
                                                                 
 dense_58 (Dense)            (None, 3)                 78        
                                                                 
Total params: 3,053,678
Trainable params: 3,053,678
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
403/403 [========================

In [164]:
lstm_glove.summary()
lstm_glove.fit(X_train,  y_train_onehot, epochs=3, validation_data=(X_val, y_val_onehot))

Model: "sequential_75"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, None, 200)         3031000   
                                                                 
 dropout_150 (Dropout)       (None, None, 200)         0         
                                                                 
 lstm_63 (LSTM)              (None, 25)                22600     
                                                                 
 dropout_151 (Dropout)       (None, 25)                0         
                                                                 
 dense_60 (Dense)            (None, 3)                 78        
                                                                 
Total params: 3,053,678
Trainable params: 22,678
Non-trainable params: 3,031,000
_________________________________________________________________
Epoch 1/3
403/403 [===================

In [137]:
blstm_random_over.summary()
blstm_random_over.fit(X_train_over,  y_train_over_onehot, epochs=3, validation_data=(X_val, y_val_onehot))

Model: "sequential_66"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 25, 25)            378875    
                                                                 
 dropout_132 (Dropout)       (None, 25, 25)            0         
                                                                 
 bidirectional_7 (Bidirectio  (None, 50)               10200     
 nal)                                                            
                                                                 
 dropout_133 (Dropout)       (None, 50)                0         
                                                                 
 dense_53 (Dense)            (None, 3)                 153       
                                                                 
Total params: 389,228
Trainable params: 389,228
Non-trainable params: 0
_______________________________________________